## Convert all netCDF NCAR timestep files to Zarr (64,64,64), round-robin across time across FileDB nodes

In [ ]:
!pip install "xarray[complete]"

In [1]:
import xarray as xr
import os
# from dask.diagnostics import ProgressBar
# import zarr # No compute image with both xarray and zarr

<font color="orange">Don't delete the CD cell!</font>

In [1]:
%cd /home/idies/workspace/turb/data02_02/ariel-6-timestep-ncar-netCDF

/home/idies/workspace/turb/data02_02/ariel-6-timestep-ncar-netCDF


### Get target Folder list

In [3]:
# creatuple list of folder on filedb to which data should be copied
# you may want to change this to avoid certain volumes
folders=[f'/home/idies/workspace/turb/data{str(d).zfill(2)}_{str(f).zfill(2)}/zarr/'  for f in range(1,4) for d in range(1,13)]

# Avoiding 7-2 and 9-2 - they're too full as of May 2023
# folders.remove("/home/idies/workspace/turb/data02_02/zarr/ncar-zarr/")
folders.remove("/home/idies/workspace/turb/data09_02/zarr/")
folders.remove("/home/idies/workspace/turb/data07_02/zarr/") # This is already created

folders[:5]

['/home/idies/workspace/turb/data01_01/zarr/',
 '/home/idies/workspace/turb/data02_01/zarr/',
 '/home/idies/workspace/turb/data03_01/zarr/',
 '/home/idies/workspace/turb/data04_01/zarr/',
 '/home/idies/workspace/turb/data05_01/zarr/']

In [ ]:
folder_path = '/home/idies/workspace/turb/data02_02/ariel-6-timestep-ncar-netCDF'

i = 0

# Possible Parallel implementation:
# Ariel: seems like contention on data_xr = xr.open_dataset(file_name)
#     bcs all source data live on data02_02
for file_name in os.listdir(folder_path):
    if os.path.isfile(os.path.join(folder_path, file_name)):
        # https://github.com/pangeo-data/pangeo/issues/150
        # with dask.config.set(get=dask.threaded.get):
#         with ProgressBar():
        data_xr = xr.open_dataset(file_name)
    
        chunk_size_base = 64 * 2**i # Try different chunk sizes for all

        # Disable compression, set chunk size
        encoding = {variable_name: {'compressor': None, 'chunks': (chunk_size_base, chunk_size_base, chunk_size_base)} for variable_name in data_xr.variables}
        
        target_dir = folders[i]

        # overwrite if exists
        data_xr.to_zarr(store=target_dir + "ncar_" + str(i) + "_" + "chunk_" + str(chunk_size_base) + ".zarr",
                        mode="w",
                       encoding = encoding)
        
        print(file_name)
        i += 1
